In [14]:
preprocessor_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/2"

In [15]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

C:\Users\bekasker\miniconda3\envs\tensorflow\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [16]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(preprocessor_url)
encoder_inputs = preprocessor(text_input)

In [17]:
encoder = hub.KerasLayer(encoder_url, trainable=True)
outputs = encoder(encoder_inputs)

pooled_output = outputs["pooled_output"]
sequence_output = outputs["sequence_output"]

In [18]:
layer = tf.keras.layers.Dropout(0.1, name='dropout')(pooled_output)
layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(layer)

model = tf.keras.Model(inputs=[text_input], outputs=[layer])

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['input_1[0][0]']                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [20]:
from tensorflow import keras
from tensorflow.keras import callbacks, metrics 

In [21]:
METRICS = [
    metrics.BinaryAccuracy(name='accuracy'),
    metrics.Precision(name='precision'),
    metrics.Recall(name='recall')
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-5), 
    loss='binary_crossentropy', 
    metrics=METRICS
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    min_delta=1e-4,
    patience=3,
    verbose=1,
    restore_best_weights=True
)

In [23]:
history = model.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=5,
    verbose=1,
    callbacks=[early_stopping]
)

Epoch 1/5
656/656 [==============================] - 263s 402ms/step - loss: 0.2708 - accuracy: 0.8888 - precision: 0.8886 - recall: 0.8891 - val_loss: 0.2298 - val_accuracy: 0.9101 - val_precision: 0.8864 - val_recall: 0.9408
Epoch 2/5
656/656 [==============================] - 265s 405ms/step - loss: 0.2177 - accuracy: 0.9104 - precision: 0.9081 - recall: 0.9133 - val_loss: 0.2013 - val_accuracy: 0.9210 - val_precision: 0.9160 - val_recall: 0.9270
Epoch 3/5
656/656 [==============================] - 267s 408ms/step - loss: 0.1751 - accuracy: 0.9335 - precision: 0.9307 - recall: 0.9368 - val_loss: 0.1974 - val_accuracy: 0.9281 - val_precision: 0.9087 - val_recall: 0.9519
Epoch 4/5
656/656 [==============================] - 270s 411ms/step - loss: 0.1447 - accuracy: 0.9451 - precision: 0.9443 - recall: 0.9460 - val_loss: 0.2016 - val_accuracy: 0.9257 - val_precision: 0.9086 - val_recall: 0.9466
Epoch 5/5
656/656 [==============================] - 263s 401ms/step - loss: 0.1156 - accura